In [49]:
# Import libs
import sys
import matplotlib.pyplot as plt
from docplex.mp.model import Model
from docplex.util.environment import get_environment
from docplex.mp.relax_linear import LinearRelaxer
import itertools

try:
    import docplex.mp
except:
    raise Exception('Please install docplex. See https://pypi.org/project/docplex/')

In [50]:
# ACEITES
A = 'A'
B = 'B'
C = 'C'
D = 'D'
E = 'E'
ACEITES = [A,B,C,D,E]
CONST_m_INDICADORAS = 0.01
CONST_M_INDICADORAS = 1000
# PRECIOS
PRECIO_ACEITE_COMESTIBLE = 150
PRECIO_ACEITE_CRUDO = {
    A: 110,
    B: 120,
    C: 130,
    D: 110,
    E: 115
}

# REFINADO
CONST_TIEMPO_REFINADO = 2 # definir

# DUREZAS
MIN_DUREZA = 3
MAX_DUREZA = 6
DUREZA_ACEITE_CRUDO = {
    A: 8.8,
    B: 6.1,
    C: 2.0,
    D: 4.2,
    E: 5.0
}

# COSTO MANTENIMIENTO
COSTO_MANTENIMIENTO = {
    1: 5000,
    2: 8000,
    3: 9500,
    4: 1000
}
LIMITES_INTERVALOS = [0, 100, 200, 500, 500]
CANTIDAD_INTERVALOS = 4
CONST_m_COTA_INFERIOR_INTERVALO = 0.1
CONST_M_COTA_SUPERIOR_INTERVALO = 20000

# CAPACIDADES
CAPACIDAD_LINEA_AB = 200
CAPACIDAD_LINEA_CDE = 250

# CONSTANTE DE MEZCLA
CANTIDAD_MIN_ACEITES_EN_MEZCLA = 2
CANTIDAD_MAX_ACEITES_EN_MEZCLA = 3
MINIMO_ACEITE_CRUDO = 20

In [51]:
mdl = Model(name="Ejericicio 3.6: Aceites")

# VARIABLES DE DECISION
producto_fabricado = mdl.continuous_var(name = "producto_final")

aceite_comprado = {tipo: mdl.continuous_var(name=f'aceite_comprado_{tipo}') for tipo in ACEITES}
se_usa_aceite = {tipo: mdl.binary_var(name=f'se_usa_aceite_{tipo}') for tipo in ACEITES}

intervalo_usado = {intervalo: mdl.binary_var(name=f'intervalo_{intervalo}_usado') for intervalo in range(1, 1+CANTIDAD_INTERVALOS)}
tiempo_refinamiento_intervalo = {intervalo: mdl.continuous_var(name=f'tiempo_refinamiento_intervalo_{intervalo}') for intervalo in range(1, 1+CANTIDAD_INTERVALOS)}
tiempo_refinamiento = mdl.continuous_var(name = "tiempo_total_de_refinamiento")

# RESTRICCIONES

# DISPONIBILIDAD DE PLANTA DE PRODUCCION
mdl.add_constraint(aceite_comprado[A] + aceite_comprado[B] <= CAPACIDAD_LINEA_AB)
mdl.add_constraint(aceite_comprado[C] + aceite_comprado[D] + aceite_comprado[E] <= CAPACIDAD_LINEA_CDE)

# INDICADORAS
for tipo in ACEITES:
    mdl.add_constraint(CONST_m_INDICADORAS*se_usa_aceite[tipo] <= aceite_comprado[tipo])
    mdl.add_constraint(aceite_comprado[tipo] <= CONST_M_INDICADORAS*se_usa_aceite[tipo])

# DUREZA
mdl.add_constraint(producto_fabricado == mdl.sum(aceite_comprado[tipo] for tipo in ACEITES))

mdl.add_constraint(MIN_DUREZA*producto_fabricado <= mdl.sum(DUREZA_ACEITE_CRUDO[tipo] * aceite_comprado[tipo] for tipo in ACEITES))
mdl.add_constraint(mdl.sum(DUREZA_ACEITE_CRUDO[tipo] * aceite_comprado[tipo] for tipo in ACEITES) <= MAX_DUREZA*producto_fabricado) 


# LIMITES DE ACEITES CRUDOS EN LA MEZCLA
mdl.add_constraint(CANTIDAD_MIN_ACEITES_EN_MEZCLA <= mdl.sum(se_usa_aceite[tipo] for tipo in ACEITES))
mdl.add_constraint(CANTIDAD_MAX_ACEITES_EN_MEZCLA >= mdl.sum(se_usa_aceite[tipo] for tipo in ACEITES))

# Minimo de cada aceite crudo en mezcla si se usa
for tipo in ACEITES:
    mdl.add_constraint(aceite_comprado[tipo] >= MINIMO_ACEITE_CRUDO * se_usa_aceite[tipo])

# SI A o B entonces C
mdl.add_constraint(se_usa_aceite[A] - se_usa_aceite[C] <= 0)
mdl.add_constraint(se_usa_aceite[B] - se_usa_aceite[C] <= 0)

# COSTOS DE MANTENIMIENTO
mdl.add_constraint(tiempo_refinamiento == CONST_TIEMPO_REFINADO*producto_fabricado)
mdl.add_constraint(tiempo_refinamiento == mdl.sum(tiempo_refinamiento_intervalo[i] for i in range(1, 1+CANTIDAD_INTERVALOS)))

mdl.add_constraint(LIMITES_INTERVALOS[0] <= tiempo_refinamiento_intervalo[1])
mdl.add_constraint((LIMITES_INTERVALOS[1] - CONST_m_COTA_INFERIOR_INTERVALO) * intervalo_usado[1] >= tiempo_refinamiento_intervalo[1])

mdl.add_constraint(LIMITES_INTERVALOS[1]*intervalo_usado[2] <= tiempo_refinamiento_intervalo[2])
mdl.add_constraint((LIMITES_INTERVALOS[2] - CONST_m_COTA_INFERIOR_INTERVALO)*intervalo_usado[2] >= tiempo_refinamiento_intervalo[2])

mdl.add_constraint(LIMITES_INTERVALOS[2]*intervalo_usado[3] <= tiempo_refinamiento_intervalo[3])
mdl.add_constraint((LIMITES_INTERVALOS[3] - CONST_m_COTA_INFERIOR_INTERVALO) * intervalo_usado[3] >= tiempo_refinamiento_intervalo[3])

mdl.add_constraint(LIMITES_INTERVALOS[3]*intervalo_usado[4] <= tiempo_refinamiento_intervalo[4])
mdl.add_constraint(CONST_M_COTA_SUPERIOR_INTERVALO*intervalo_usado[4] >= tiempo_refinamiento_intervalo[4])

mdl.add_constraint(mdl.sum(intervalo_usado[intervalo] for intervalo in range(1, 1+CANTIDAD_INTERVALOS)) <= 1)


# FUNCION OBJETIVO
ganancia = PRECIO_ACEITE_COMESTIBLE*producto_fabricado
costo_aceite = mdl.sum(PRECIO_ACEITE_CRUDO[tipo] * aceite_comprado[tipo] for tipo in ACEITES)
costo_maquina = mdl.sum(intervalo_usado[intervalo] * COSTO_MANTENIMIENTO[intervalo] for intervalo in COSTO_MANTENIMIENTO.keys())
beneficio = ganancia - costo_aceite - costo_maquina
mdl.maximize(beneficio)




solution = mdl.solve()

In [52]:
# Imprimir la solución
if solution:
    print(f'BENEFICIO:')
    print(f' - {mdl.objective_value}$')
    
else:
    print("No se encontro solución.")

BENEFICIO:
 - 16000.0$
